# Trainining on AI Platform with scikit-learn

In [1]:
import os
import time

## Configure environment

*You need to walk through the `local-experimentation.ipynb` notebook to create training and validation datasets.*


In [2]:
PROJECT_ID = !(gcloud config get-value core/project)
ARTIFACT_STORE = 'gs://{}-artifact-store'.format(PROJECT_ID[0])

TRAINING_DATA_PATH = '{}/datasets/training.csv'.format(ARTIFACT_STORE)
TESTING_DATA_PATH = '{}/datasets/testing.csv'.format(ARTIFACT_STORE)
REGION = "us-central1"
JOBDIR_BUCKET = '{}/jobs'.format(ARTIFACT_STORE)

## Create a training application package
### Create a training module

In [3]:
TRAINING_APP_FOLDER = '../training_app/trainer'

os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)
!touch $TRAINING_APP_FOLDER/__init__.py

In [4]:
%%writefile $TRAINING_APP_FOLDER/train.py

import logging
import os
import subprocess
import sys

import fire
import joblib
import numpy as np
import pandas as pd

from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import Ridge
from sklearn.manifold import TSNE 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


def train(job_dir, data_path, n_features_options, l2_reg_options):
    
  # Load data from GCS
  df_train = pd.read_csv(data_path)

  y = df_train.octane
  X = df_train.drop('octane', axis=1)
    
  # Configure a training pipeline
  pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('reduce_dim', PCA()),
    ('regress', Ridge())
  ])

  # Configure a parameter grid
  param_grid = [
    {
      'reduce_dim__n_components': n_features_options,
      'regress__alpha': l2_reg_options
    }
  ]

  # Tune hyperparameters
  grid = GridSearchCV(pipeline, cv=10, n_jobs=None, param_grid=param_grid, scoring='neg_mean_squared_error', iid=False)
  grid.fit(X, y)

  logging.info("Best estimator: {}".format(grid.best_params_))
  logging.info("Best score: {}".format(grid.best_score_))
    
  # Retrain the best model on a full dataset
  best_estimator = grid.best_estimator_
  trained_pipeline = best_estimator.fit(X, y)

  # Save the model
  model_filename = 'model.joblib'
  joblib.dump(value=trained_pipeline, filename=model_filename)
  gcs_model_path = "{}/{}".format(job_dir, model_filename)
  subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
  logging.info("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
  logging.basicConfig(level=logging.INFO)
  fire.Fire(train)

Overwriting ../training_app/trainer/train.py


### Configure dependencies

In [5]:
%%writefile $TRAINING_APP_FOLDER/../setup.py

from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = ['fire', 'gcsfs']

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='My training application package.'
)

Overwriting ../training_app/trainer/../setup.py


## Submit a training job

In [17]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
SCALE_TIER = "BASIC"
MODULE_NAME = "trainer.train"
RUNTIME_VERSION = "2.1"
PYTHON_VERSION = "3.7"

N_FEATURES_OPTIONS="[2,4,6]"
L2_REG_OPTIONS="[0.1,0.2,0.3,0.5]"

In [18]:
!gcloud ai-platform jobs submit training $JOB_NAME \
--region $REGION \
--job-dir $JOBDIR_BUCKET/$JOB_NAME \
--package-path $TRAINING_APP_FOLDER \
--module-name $MODULE_NAME \
--scale-tier $SCALE_TIER \
--python-version $PYTHON_VERSION \
--runtime-version $RUNTIME_VERSION \
-- \
--data_path $TRAINING_DATA_PATH \
--n_features_options $N_FEATURES_OPTIONS \
--l2_reg_options $L2_REG_OPTIONS

Job [JOB_20200312_220502] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20200312_220502

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20200312_220502
jobId: JOB_20200312_220502
state: QUEUED


In [19]:
!gcloud ai-platform jobs describe $JOB_NAME
!gcloud ai-platform jobs stream-logs $JOB_NAME

createTime: '2020-03-12T22:05:05Z'
etag: zlqtE1P-YA8=
jobId: JOB_20200312_220502
state: PREPARING
trainingInput:
  args:
  - --data_path
  - gs://mlops-dev-100-artifact-store/datasets/training.csv
  - --n_features_options
  - '[2,4,6]'
  - --l2_reg_options
  - '[0.1,0.2,0.3,0.5]'
  jobDir: gs://mlops-dev-100-artifact-store/jobs/JOB_20200312_220502
  packageUris:
  - gs://mlops-dev-100-artifact-store/jobs/JOB_20200312_220502/packages/9438eaff6f9be25fdb8e531073f82df703d5da5e15517bb77b8a4735aabe2ba9/trainer-0.1.tar.gz
  pythonModule: trainer.train
  pythonVersion: '3.7'
  region: us-central1
  runtimeVersion: '2.1'
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/JOB_20200312_220502?project=mlops-dev-100

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2FJOB_20200312_220502&project=mlops-dev-100
INFO	2020-03-12 22:05:05 +0000	service		Validating job requirements...
INFO	2020-03-12 22:05:05 +0000	se

### Copy the trained model to model repo

In [20]:
MODEL_NAME = "octane-regressor-r14"

!gsutil cp $JOBDIR_BUCKET/$JOB_NAME/model.joblib $ARTIFACT_STORE/models/$MODEL_NAME/model.joblib

Copying gs://mlops-dev-100-artifact-store/jobs/JOB_20200312_220502/model.joblib [Content-Type=application/octet-stream]...
/ [1 files][ 33.1 KiB/ 33.1 KiB]                                                
Operation completed over 1 objects/33.1 KiB.                                     


## Download and test the model

In [21]:
LOCAL_PATH = '/tmp/model.joblib'

!gsutil cp $ARTIFACT_STORE/models/$MODEL_NAME/model.joblib $LOCAL_PATH 

Copying gs://mlops-dev-100-artifact-store/models/octane-regressor-r14/model.joblib...
/ [1 files][ 33.1 KiB/ 33.1 KiB]                                                
Operation completed over 1 objects/33.1 KiB.                                     


In [22]:
from sklearn.externals import joblib
import pandas as pd

df_test = pd.read_csv(TESTING_DATA_PATH)
predictor = joblib.load(LOCAL_PATH)

y = df_test.octane
X = df_test.drop('octane', axis=1)

y_hat = predictor.predict(X)
print(list(zip(y, y_hat)))

[(88.0, 88.27798345874527), (87.6, 87.36663271824897), (85.5, 85.49529039393362), (87.15, 87.39725389385443), (83.4, 83.6765741658715), (88.1, 87.99599189557136)]
